In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime, timedelta
import random

In [2]:
df = pd.read_csv(r'C:\Users\91988\OneDrive\Documents\abc_gaming_dummy_dataset.ipynb.csv')

In [3]:
df.head()

,Player_ID,Timestamp,Action_Type,Amount,Games_Played
0,P082,2023-10-14 08:25:56,Deposit,4512,0
1,P029,2023-10-05 23:24:55,Deposit,9435,0
2,P012,2023-10-02 13:01:46,Game,0,1
3,P012,2023-10-25 12:46:53,Deposit,934,0
4,P072,2023-10-27 10:54:10,Deposit,7373,0


In [4]:
random.seed(42)
np.random.seed(42)

generate list for player id

In [5]:
player_ids = [f"P{str(i).zfill(3)}" for i in range(1, 101)]

generate transaction data for october

In [6]:
actions = ['Deposit', 'Withdrawal', 'Game']
data = []

In [7]:
for player in player_ids:
    num_records = random.randint(15, 40)  # each player has 15-40 activities
    for _ in range(num_records):
        action = random.choice(actions)
        date = datetime(2023, 10, 1) + timedelta(days=random.randint(0, 30), hours=random.randint(0, 23), minutes=random.randint(0, 59))
        amount = 0
        games_played = 0

        if action == 'Deposit':
            amount = random.randint(100, 5000)
        elif action == 'Withdrawal':
            amount = random.randint(50, 3000)
        elif action == 'Game':
            games_played = random.randint(1, 5)

        data.append([player, date, action, amount, games_played])

Create dataframe

In [8]:
df = pd.DataFrame(data, columns=['Player_ID', 'Timestamp', 'Action_Type', 'Amount', 'Games_Played'])

In [9]:
df.head()

,Player_ID,Timestamp,Action_Type,Amount,Games_Played
0,P001,2023-10-01 23:17:00,Deposit,2106,0
1,P001,2023-10-05 23:06:00,Deposit,4567,0
2,P001,2023-10-19 13:02:00,Deposit,344,0
3,P001,2023-10-07 07:32:00,Deposit,317,0
4,P001,2023-10-07 22:41:00,Game,0,5


convert timestamp to date time

In [10]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

to calculate loyalty points

In [11]:
def calculate_loyalty_points(data):
    grouped = data.groupby('Player_ID').agg(
        total_deposit=('Amount', lambda x: x[data.loc[x.index, 'Action_Type'] == 'Deposit'].sum()),
        deposit_count=('Action_Type', lambda x: (x == 'Deposit').sum()),
        total_withdrawal=('Amount', lambda x: x[data.loc[x.index, 'Action_Type'] == 'Withdrawal'].sum()),
        withdrawal_count=('Action_Type', lambda x: (x == 'Withdrawal').sum()),
        total_games=('Games_Played', 'sum')
    ).reset_index()

    grouped['extra_points'] = 0.001 * (grouped['deposit_count'] - grouped['withdrawal_count']).clip(lower=0)
    grouped['loyalty_points'] = (
        0.01 * grouped['total_deposit'] +
        0.005 * grouped['total_withdrawal'] +
        grouped['extra_points'] +
        0.2 * grouped['total_games']
    )

    return grouped[['Player_ID', 'loyalty_points']].sort_values(by='loyalty_points', ascending=False)

Define slots

In [12]:
slots = {
    '2nd Oct S1': (datetime(2023, 10, 2, 0), datetime(2023, 10, 2, 12)),
    '16th Oct S2': (datetime(2023, 10, 16, 12), datetime(2023, 10, 17, 0)),
    '18th Oct S1': (datetime(2023, 10, 18, 0), datetime(2023, 10, 18, 12)),
    '26th Oct S2': (datetime(2023, 10, 26, 12), datetime(2023, 10, 27, 0)),
}

calculate loyalty point per slot

In [13]:
slot_results = {}
for slot_name, (start, end) in slots.items():
    slot_data = df[(df['Timestamp'] >= start) & (df['Timestamp'] < end)]
    slot_results[slot_name] = calculate_loyalty_points(slot_data)

show top 5

In [14]:
{slot: data.head(5) for slot, data in slot_results.items()}

{'2nd Oct S1':    Player_ID  loyalty_points
 5       P014          61.372
 0       P002          47.471
 8       P018          38.665
 12      P028          38.641
 6       P015          38.441,
 '16th Oct S2':    Player_ID  loyalty_points
 13      P052          49.850
 12      P051          49.201
 11      P050          46.791
 20      P062          45.281
 32      P088          42.391,
 '18th Oct S1':    Player_ID  loyalty_points
 17      P046          74.112
 5       P012          47.001
 26      P065          40.191
 37      P097          38.496
 0       P001          30.621,
 '26th Oct S2':    Player_ID  loyalty_points
 15      P035          49.931
 30      P079          49.545
 29      P078          47.731
 27      P068          44.061
 3       P010          43.085}

monthly loyalty points per player

In [15]:
monthly_loyalty = calculate_loyalty_points(df)


Merge additional data: total games played for tie-breaker

In [16]:
games_data = df[df['Action_Type'] == 'Game'].groupby('Player_ID')['Games_Played'].sum().reset_index()
monthly_loyalty = pd.merge(monthly_loyalty, games_data, on='Player_ID', how='left').fillna(0)


In [17]:
monthly_loyalty['Rank'] = monthly_loyalty.sort_values(
    by=['loyalty_points', 'Games_Played'], ascending=[False, False]
).reset_index(drop=True).index + 1


A3 average Deposit

In [18]:
avg_deposit = df[df['Action_Type'] == 'Deposit']['Amount'].mean()


In [19]:
deposits_per_user = df[df['Action_Type'] == 'Deposit'].groupby('Player_ID')['Amount'].sum()
avg_deposit_per_user = deposits_per_user.mean()


A5 avg no of game played per user

A4  Average Deposit Amount per User in a Month

In [20]:
games_per_user = df[df['Action_Type'] == 'Game'].groupby('Player_ID')['Games_Played'].sum()
avg_games_per_user = games_per_user.mean()

top 5 players and avgs

In [21]:
monthly_summary = monthly_loyalty.sort_values(by='Rank').head(5)
{
    "Top 5 Players by Loyalty Points": monthly_summary,
    "Average Deposit Amount": avg_deposit,
    "Average Deposit per User": avg_deposit_per_user,
    "Average Games Played per User": avg_games_per_user
}


{'Top 5 Players by Loyalty Points':   Player_ID  loyalty_points  Games_Played  Rank
 0      P074         625.423            34     1
 1      P002         558.616            39     2
 2      P043         505.907            34     3
 3      P035         498.183            23     4
 4      P052         477.687            39     5,
 'Average Deposit Amount': np.float64(2557.6083244397014),
 'Average Deposit per User': np.float64(23964.79),
 'Average Games Played per User': np.float64(30.21)}

                PART B

Bonus for a player = (Player’s loyalty points / Total loyalty points of top 50 players) * 50,000

Calculate loyalty points for each row

In [22]:
print(df.columns)

Index(['Player_ID', 'Timestamp', 'Action_Type', 'Amount', 'Games_Played'], dtype='object')


Create new columns based on action type

In [23]:

df['Deposit_Amount'] = df.apply(lambda x: x['Amount'] if x['Action_Type'] == 'Deposit' else 0, axis=1)
df['Withdrawal_Amount'] = df.apply(lambda x: x['Amount'] if x['Action_Type'] == 'Withdraw' else 0, axis=1)
df['Num_Deposits'] = df['Action_Type'].apply(lambda x: 1 if x == 'Deposit' else 0)
df['Num_Withdrawals'] = df['Action_Type'].apply(lambda x: 1 if x == 'Withdraw' else 0)


Group by Player to get total values:

In [24]:
player_loyalty_df = df.groupby('Player_ID').agg({
    'Deposit_Amount': 'sum',
    'Withdrawal_Amount': 'sum',
    'Num_Deposits': 'sum',
    'Num_Withdrawals': 'sum',
    'Games_Played': 'sum'
}).reset_index()

Calculate Loyalty Points

In [25]:
player_loyalty_df['Loyalty_Points'] = (
    0.01 * player_loyalty_df['Deposit_Amount'] +
    0.005 * player_loyalty_df['Withdrawal_Amount'] +
    0.001 * (player_loyalty_df['Num_Deposits'] - player_loyalty_df['Num_Withdrawals']).clip(lower=0) +
    0.2 * player_loyalty_df['Games_Played']
)

Now sort and get Top 50 players:

In [26]:
top_50 = player_loyalty_df.sort_values(by=['Loyalty_Points', 'Games_Played'], ascending=[False, False]).head(50)

Bonus Distribution (Rs 50,000)

In [27]:
total_loyalty_top_50 = top_50['Loyalty_Points'].sum()
top_50['Bonus_Amount'] = (top_50['Loyalty_Points'] / total_loyalty_top_50) * 50000

In [39]:
top_50.to_csv('Top_50_Loyalty_Leaderboard_with_Bonus.csv', index=False)
print("Bonus-distributed leaderboard saved as 'Top_50_Loyalty_Leaderboard_with_Bonus.csv'")

Bonus-distributed leaderboard saved as 'Top_50_Loyalty_Leaderboard_with_Bonus.csv'


                                  PART C

Add dummy columns with default values

In [29]:
# Add dummy columns with default values (for testing)
df['Num_Deposits'] = df.get('Num_Deposits', 3)  # Replace 3 with your default
df['Num_Withdrawals'] = df.get('Num_Withdrawals', 2)  # Replace 2 with your default
df['Games_Won'] =  df.get('Games_Won', 10)  # Assume player won 10 games
df['Login_Streak_Days'] = df.get('Login_Streak_Days', 5)  # Assume 5-day streak


In [30]:
df['Loyalty_Points'] = (
    0.008 * df['Deposit_Amount'] +
    0.004 * df['Withdrawal_Amount'] +
    0.001 * (df['Num_Deposits'] - df['Num_Withdrawals']).clip(lower=0) +
    0.15 * df['Games_Played'] +
    0.1 * df['Games_Won'] +
    0.02 * df['Login_Streak_Days']
)

 Sort by loyalty points and games played

In [36]:
top_50.to_csv("Top_50_Hybrid_Bonus_Leaderboard.csv", index=False)

In [31]:
top_50 = df.sort_values(by=['Loyalty_Points', 'Games_Played'], ascending=[False, False]).head(50)


In [32]:
top_50.reset_index(drop=True, inplace=True)
top_50['Rank'] = top_50.index + 1

In [33]:
top_50.to_csv('Top_50_Loyalty_Leaderboard.csv', index=False)

In [37]:
import pandas as pd
bonus_df = pd.read_csv("Top_50_Hybrid_Bonus_Leaderboard.csv")
bonus_df.head()

,Player_ID,Timestamp,Action_Type,Amount,Games_Played,Deposit_Amount,Withdrawal_Amount,Num_Deposits,Num_Withdrawals,Games_Won,Login_Streak_Days,Loyalty_Points,Rank
0,P058,2023-10-10 12:04:00,Deposit,5000,0,5000,0,1,0,10,5,41.101,1
1,P080,2023-10-30 15:17:00,Deposit,4997,0,4997,0,1,0,10,5,41.077,2
2,P068,2023-10-27 14:56:00,Deposit,4995,0,4995,0,1,0,10,5,41.061,3
3,P035,2023-10-27 20:09:00,Deposit,4974,0,4974,0,1,0,10,5,40.893,4
4,P061,2023-10-20 02:03:00,Deposit,4967,0,4967,0,1,0,10,5,40.837,5


In [34]:

print("Top 50 leaderboard has been saved as 'Top_50_Loyalty_Leaderboard.csv'")

Top 50 leaderboard has been saved as 'Top_50_Loyalty_Leaderboard.csv'
